In [1]:
import pandas as pd

from ddeserts.annotate import add_all_stat_columns
from ddeserts.annotate import add_geo_columns
from ddeserts.annotate import with_columns_sorted
from ddeserts.load import load_cvap_data
from ddeserts.load import load_charter_cities

In [2]:
# load the data
pre_filter = lambda line: ('California' in line and 'CDP' not in line)

df = load_cvap_data('Place', pre_filter=pre_filter)
len(df)

485

In [3]:
# add geo columns and post-filter
add_geo_columns(df)
df = df[(df['state'] == 'California') & df['geotype'].isin(['city', 'town'])]
len(df)

482

In [4]:
# add charter city data
charter_cities = load_charter_cities()
df['hascharter'] = df['name'].isin(charter_cities)

In [5]:
# add stat columns and sort
add_all_stat_columns(df)
df = with_columns_sorted(df)
df

geoid                        geoname geotype  \
table line                                                           
Place 25130  16000US0600296      Adelanto city, California    city   
      25156  16000US0600394  Agoura Hills city, California    city   
      25221  16000US0600562       Alameda city, California    city   
      25247  16000US0600674        Albany city, California    city   
      25286  16000US0600884      Alhambra city, California    city   
...                     ...                            ...     ...   
      44786  16000US0686930    Yountville city, California    city   
      44799  16000US0686944         Yreka city, California    city   
      44812  16000US0686972     Yuba City city, California    city   
      44825  16000US0687042       Yucaipa city, California    city   
      44838  16000US0687056  Yucca Valley town, California    town   

             hascharter          name       state  adu_est  adu_moe  cit_est  \
table line                                                                     
Place 25130        True      Adelanto  California    21995      653    29610   
      25156       False  Agoura Hills  California    16140      393    19320   
      25221        True       Alameda  California    62565      524    72155   
      25247        True        Albany  California    14665      241    16290   
      25286        True      Alhambra  California    70315      496    69830   
...                 ...           ...         ...      ...      ...      ...   
      44786       False    Yountville  California     2760       81     2940   
      44799       False         Yreka  California     5660      180     7500   
      44812       False     Yuba City  California    49455      404    58480   
      44825       False       Yucaipa  California    40175      646    50875   
      44838       False  Yucca Valley  California    16730      502    21165   

             cit_moe  ...  p_tot_ind_est  p_tot_ind_moe  p_tot_oth_est  \
table line            ...                                                
Place 25130      532  ...       0.000297       0.000446       0.025698   
      25156      468  ...       0.000000       0.001123       0.035306   
      25221      810  ...       0.002611       0.001251       0.061895   
      25247      629  ...       0.006312       0.003347       0.060843   
      25286      975  ...       0.002894       0.001400       0.015004   
...              ...  ...            ...            ...            ...   
      44786       41  ...       0.003350       0.004744       0.031826   
      44799      107  ...       0.046958       0.012007       0.076720   
      44812      771  ...       0.006390       0.002082       0.049312   
      44825      470  ...       0.004025       0.003771       0.017504   
      44838      199  ...       0.006938       0.003714       0.025671   

             p_tot_oth_moe  p_tot_pac_est  p_tot_pac_moe  p_tot_wht_est  \
table line                                                                
Place 25130       0.047564       0.001485       0.003986       0.120915   
      25156       0.048639       0.000000       0.001123       0.747748   
      25221       0.026146       0.003439       0.001736       0.430846   
      25247       0.049723       0.004292       0.003091       0.444332   
      25286       0.022648       0.002481       0.002760       0.088605   
...                    ...            ...            ...            ...   
      44786       0.103451       0.000000       0.004047       0.737018   
      44799       0.060412       0.014550       0.009651       0.712302   
      44812       0.019401       0.004435       0.001147       0.425318   
      44825       0.037777       0.000936       0.001125       0.588880   
      44838       0.078553       0.010407       0.011406       0.661887   

             p_tot_wht_moe  p_wht_dis_est  p_wht_dis_moe  
table line                                                
Place 25130  

In [6]:
df.to_csv('output/ca-cities.csv')

In [19]:
dg = df[['name', 'hascharter', 'underrep_score_est', 'underrep_score_moe', 'underrep_his_est', 'underrep_his_moe']]
dg[dg['hascharter']].sort_values(by=['underrep_his_est'], ascending=False)[:30]

name  hascharter  underrep_score_est  \
table line                                                        
Place 33788           King City        True           18.525080   
      40860          San Rafael        True           14.031275   
      40340             Salinas        True           13.415319   
      40990         Santa Maria        True           13.411184   
      40522           Sand City        True           11.536708   
      39742            Richmond        True           11.341280   
      28900             Compton        True           11.282848   
      34984             Lindsay        True           10.212312   
      43928         Watsonville        True           10.163313   
      25546             Anaheim        True            9.878711   
      41354             Shafter        True            9.674508   
      33268           Inglewood        True           10.094704   
      29732  Desert Hot Springs        True            9.783682   
      37233                Napa        True            9.765609   
      40899           Santa Ana        True            9.331430   
      39651        Redwood City        True           10.739218   
      40912       Santa Barbara        True            9.177350   
      35400         Los Angeles        True            9.245349   
      41042          Santa Rosa        True            9.141798   
      39157         Porterville        True            8.059844   
      40756          San Marcos        True            8.281746   
      37740             Oakland        True            8.169022   
      43148              Tulare        True            7.330627   
      30915              Exeter        True            7.230165   
      36284              Merced        True            6.749168   
      25130            Adelanto        True            7.301530   
      40457      San Bernardino        True            7.155974   
      43720               Vista        True            7.019546   
      35309          Long Beach        True            6.810683   
      40795           San Mateo        True            8.621668   

             underrep_score_moe  underrep_his_est  underrep_his_moe  
table line                                                           
Place 33788           26.810939          0.185251          0.265943  
      40860            3.083649          0.134411          0.025662  
      40340            4.045495          0.134153          0.040438  
      40990            4.910961          0.134112          0.049025  
      40522           63.063690          0.115367          0.413185  
      39742            3.872577          0.113413          0.030502  
      28900            4.246812          0.112620          0.041824  
      34984           20.549318          0.102123          0.204259  
      43928            7.549338          0.101633          0.074798  
      25546            2.102090          0.098787          0.020744  
      41354           13.202444          0.096745          0.131389  
      33268            3.568191          0.096371          0.034361  
      29732            9.661735          0.094919          0.089846  
      37233            2.407935          0.093939          0.022785  
      40899            2.797543          0.093314          0.027952  
      39651            3.397913          0.088934          0.028314  
      40912            2.575011          0.085192          0.023823  
      35400            0.497660          0.084399          0.004435  
      41042            1.917752          0.080334          0.016964  
      39157            6.465163          0.079973          0.061240  
      40756            3.644616          0.077902          0.029060  
      37740            1.253510          0.072734          0.008720  
      43148            5.849980          0.072679          0.056661  
      30915           13.652646          0.072302          0.106583  
      36284            4.356323          0.067492          0.0

In [20]:
dg = df[['name', 'hascharter', 'underrep_score_est', 'underrep_score_moe', 'underrep_asn_est', 'underrep_asn_moe']]
dg[dg['hascharter']].sort_values(by=['underrep_asn_est'], ascending=False)[:30]

name  hascharter  underrep_score_est  \
table line                                                   
Place 42407      Sunnyvale        True           13.343859   
      40925    Santa Clara        True           11.707320   
      25247         Albany        True            9.845989   
      33346         Irvine        True            8.173348   
      40873      San Ramon        True            8.128540   
      37077  Mountain View        True           12.344362   
      43005       Torrance        True            6.287712   
      25702        Arcadia        True            5.906080   
      29407        Cypress        True            6.013816   
      26443       Berkeley        True            5.553265   
      38260      Palo Alto        True            5.475601   
      31253         Folsom        True            5.290007   
      25286       Alhambra        True            4.481149   
      28198       Cerritos        True            3.496952   
      40665       San Jose        True            6.146760   
      40795      San Mateo        True            8.621668   
      38442       Pasadena        True            7.254223   
      27249        Burbank        True            3.159567   
      40704    San Leandro        True            5.871742   
      35348   Los Alamitos        True            3.290339   
      25221        Alameda        True            3.094760   
      35192     Loma Linda        True            3.287591   
      40535      San Diego        True            6.206068   
      40951     Santa Cruz        True            5.370716   
      29342    Culver City        True            4.692228   
      42706    Temple City        True            3.620649   
      40587  San Francisco        True            3.920567   
      39651   Redwood City        True           10.739218   
      40158      Roseville        True            2.749805   
      27223     Buena Park        True            6.565256   

             underrep_score_moe  underrep_asn_est  underrep_asn_moe  
table line                                                           
Place 42407            3.451546          0.127184          0.028594  
      40925            3.006339          0.115677          0.027061  
      25247            6.920116          0.086147          0.050896  
      33346            2.284577          0.081733          0.022533  
      40873            3.314207          0.081285          0.031232  
      37077            3.699756          0.075435          0.029559  
      43005            2.483337          0.062877          0.020122  
      25702            6.004182          0.059061          0.057825  
      29407            4.721332          0.053302          0.034225  
      26443            2.002171          0.051543          0.015643  
      38260            3.272833          0.050138          0.029491  
      31253            2.262561          0.049376          0.016663  
      25286            3.352808          0.044811          0.032023  
      28198            5.045811          0.034970          0.043923  
      40665            0.986339          0.030903          0.007441  
      40795            3.273879          0.027893          0.021692  
      38442            2.675732          0.027117          0.015368  
      27249            2.964328          0.026942          0.015877  
      40704            3.989326          0.026163          0.030225  
      35348            8.770164          0.024667          0.039355  
      25221            3.309004          0.023694          0.025713  
      35192           11.268753          0.023642          0.065237  
      40535            0.763788          0.023313          0.004598  
      40951            2.605767          0.022747          0.012389  
      29342            4.811632          0.022726          0.028890  
      42706            6.996438          0.022329          0.060365  
      40587            0.499796          0.019892          0.004221  
      3